# Empathic Zoom

Este script utiliza Selenium y la librería Deep Face para detectar las caras de los participantes en una reunión de Zoom web y mostrar las emociones de la reunión.

Para ello, el script abrirá una sesión de Chrome automatizada con Selenium, accederá a la reunión de Zoom utilizando la URL proporcionada, y luego utilizará Deep Face para analizar las expresiones faciales de los participantes.

*(Podemos llegar a usar OpenCV para analizar, ademas de las emociones, los gestos corporales)*

*Por ejemplo:*
- Detección de Movimiento y Postura: Utilizar algoritmos de visión por computadora para detectar el movimiento de las personas y su postura durante la reunión. Un aumento en el movimiento o cambios en la postura pueden indicar un mayor nivel de interés o participación.
- Reconocimiento de Gestos Faciales: Analizar los gestos faciales de los participantes para detectar sonrisas, fruncir el ceño, miradas de atención, entre otros.
- Participación en la Conversación: Analizar la participación verbal de las personas en la reunión, como la cantidad de tiempo que hablan, la frecuencia con la que hacen preguntas o comentarios, etc.

## Importar Librerías

In [2]:
# Rutas
from pyprojroot.here import here

# Selenium 
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Detección emociones
from collections import Counter
from deepface import DeepFace
import cv2

**Importante** a partir de selenium 4 ya no es necesario descargar el driver (ver version instalada).

Para la ejecución de la notebook contamos con la 4.21.0.

[Link](https://stackoverflow.com/questions/76461596/unable-to-use-selenium-webdriver-getting-two-exceptions).

## Definir rutas

In [3]:
# Rutas 
SCREENSHOTS_PATH = here() / "screenshots" 
SCREENSHOT_FILE = SCREENSHOTS_PATH / "screenshot.png"
FACE_DETECTED_FILE = SCREENSHOTS_PATH / "faces_detected.png"

# Parámetro para prueba en mi máquina local 
# Para la ejecucion en otra máquina cambiar el parámetro TEST a False.
# O sino cambiar los valores de los parámetros NAVEGADOR, MEETING_ID, ACCESS_CODE.
TEST = True
NAVEGADOR = "edge"
MEETING_ID = 4702268490
ACCESS_CODE = "8513"

## Definir funciones para la ejecución de la notebook

In [4]:
def join_zoom_meeting():
    """Entrar en reunión de ZOOM"""
    # Definir los parámetros de la reunión
    if TEST:
        print("Probando desde una reunión fija...")
        zoom_params = {"browser": NAVEGADOR, "meeting_id": MEETING_ID, "access_code": ACCESS_CODE}
    else:
        zoom_params = {"browser": input("Escribir navegador: edge/chrome..."), "meeting_id": input("Escribir ID de la reunión..."), "access_code": input("Escribir Código de Acceso de la reunión...")}
        
    browser = zoom_params.get("browser")
    meeting_id = zoom_params.get("meeting_id")
    access_code = zoom_params.get("access_code")
    user_name = "Empathic Zoom Grupo 1"
    website = f"https://zoom.us/wc/join/{meeting_id}"
    join_attempts = 0
    max_join_attempts = 5
    
    # Inicializar navegador
    if browser=="edge":
        driver = webdriver.Edge() 
    elif browser=="chrome":
        driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(website)
    time.sleep(5)
    
    # Completar parámetros de reunión de ZOOM
    while join_attempts < max_join_attempts:
        try:
            # Completar
            # Access code
            access_code_input = driver.find_element(By.XPATH, "//*[@id='input-for-pwd']")
            access_code_input.send_keys(access_code)
            time.sleep(2)
            
            # User name
            user_name_input = driver.find_element(By.XPATH, "//*[@id='input-for-name']")
            user_name_input.send_keys(user_name)
            time.sleep(2)
            
            # Unirse a la reunión
            driver.find_element(By.XPATH, "//*[@id='root']/div/div[1]/div/div[2]/button").click()
            time.sleep(5)
            
            # Configurar audio
            while True:
                try:
                    driver.find_element(By.XPATH, "//*[@id='voip-tab']/div/button").click()
                    time.sleep(2)
                    break
                except Exception as e:
                    print(f"Error: {e}")
            break
            
        except Exception as e:
            print(f"Error: {e}")
            join_attempts += 1

    if join_attempts == max_join_attempts:
        print(f"Fallo después {max_join_attempts} intentos. Yendo...")
        time.sleep(3)  
        driver.quit()
        
    return driver

def capture_screenshot(driver, filename=SCREENSHOT_FILE):
    """
    Obtener y guardar captura de pantalla con Selenium
    """
    driver.save_screenshot(filename)

def analyze_emotion(image_path=SCREENSHOT_FILE):
    """
    Analizar emociones con DeepFace
    """
    img = cv2.imread(image_path)
    return DeepFace.analyze(img, detector_backend='mtcnn', actions=['emotion']) # TODO: probar con otros modelos para detectar caras... el default no detectaba bien las caras para el archivo "screenshots\test\test_zoom_meeting.jpg"

def compute_engagement(emotion_results):
    """
    Armar índice personalizado de las emociones de la clase
    """
    engagement_scores = {
        'happy': 1,
        'surprise': 1,
        'neutral': 0.5,
        'sad': -1,
        'angry': -1,
        'fear': -1,
        'disgust': -1
    }
    total_score = 0
    for res in emotion_results:
        dominant_emotion = res.get('dominant_emotion')
        total_score += engagement_scores.get(dominant_emotion, 0)
    
    return total_score / len(emotion_results)

def save_detected_faces(deep_face_results, image_path = SCREENSHOT_FILE, faces_detected_path = FACE_DETECTED_FILE):
    """
    Identificar y marcar las caras detectadas 
    """
    # Cargar la imagen original
    image = cv2.imread(SCREENSHOT_FILE)
    
    # Dibujar rectángulos alrededor de las caras detectadas en una copia de la imagen original
    image_with_rectangles = image.copy()
    for i, face in enumerate(deep_face_results):
        x, y, w, h = face['region']['x'], face['region']['y'], face['region']['w'], face['region']['h']
        cv2.rectangle(image_with_rectangles, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Obtener la emoción dominante
        dominant_emotion = face['dominant_emotion']
        
        # Crear la etiqueta
        label = f"Cara {i} - {dominant_emotion}"
        
        # Calcular la posición de la etiqueta
        label_position = (x, y - 10 if y - 10 > 10 else y + 10)
        
        # Colocar la etiqueta en la imagen
        cv2.putText(image_with_rectangles, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
    # Guardar la imagen con los rectángulos dibujados
    cv2.imwrite(FACE_DETECTED_FILE, image_with_rectangles)

## Ejecutar Aplicación

In [4]:
# Ingresar en la reunión de zoom
driver = join_zoom_meeting() 


# Ir capturando las emociones cada 10 segundos
screenshot_nro = 0
screenshot_nro_max = 1
while screenshot_nro <= screenshot_nro_max:
    # Tomar captura de pantalla
    capture_screenshot(driver)
    
    # Obtener emociones para las caras detectadas
    if screenshot_nro == 0:
        # para el inicio del proceso
        emotion_results_0 = analyze_emotion()
        
        # Caras identificadas al inicio del proceso #TODO: ver...
        save_detected_faces(emotion_results_0, faces_detected_path = SCREENSHOTS_PATH / "faces_detected_0.png")
    
    emotion_results = analyze_emotion()
    
    # Filtrado de FOTOS (caras sin cambios)
    # Usar la clave "region" de emotion_results (ver si hacerlo con la results_0 o con la anterior)
    # Puedo comparar emotion_results y emotion_results_0 para eliminar elementos repetidos...
    
    # Mostrar los resultados por cara detectada
    print(f"\nResultados de análisis de emociones para la captura de pantalla {screenshot_nro}:\n")
    print(f"\nCaras detectadas: {len(emotion_results)}\n")
    emotions = []
    for cara_nro, emotion_result in enumerate(emotion_results):
        emotion = emotion_result['emotion']
        strongest_emotion = max(emotion, key=emotion.get)
        score = emotion[strongest_emotion]
        print(f"* CARA {cara_nro}: Emoción más fuerte: {strongest_emotion}, Score: {score}")
        dominant_emotion = emotion_result['dominant_emotion']
        emotions.append(dominant_emotion)
        

    # Obtener la moda: emocion mas frecuente
    emotion_counts = Counter(emotions)
    most_dominant_emotion = emotion_counts.most_common(1)[0][0]
    print(f"\nLa emoción más dominante es: {most_dominant_emotion}\n")    
    
    # Calcular índice personalizado de la clase
    engagement_score = compute_engagement(emotion_results)
    print(f"\nValor general del índice: {engagement_score}\n")
    
    # Caras identificadas al final del proceso
    if screenshot_nro == screenshot_nro_max:
        save_detected_faces(emotion_results)
    
    # Esperar 10 segundos
    print("\n##################\n")
    time.sleep(10)  
    
    # Pasar a la siguiente iteración
    screenshot_nro += 1
    
# cerrar driver
driver.quit() 

Probando desde una reunión fija...
Error: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='voip-tab']/div/button"}
  (Session info: MicrosoftEdge=125.0.2535.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF71CBA5DC2+61250]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF71CB306C9+206985]
	(No symbol) [0x00007FF71C93E3C7]
	(No symbol) [0x00007FF71C9852A3]
	(No symbol) [0x00007FF71C985366]
	(No symbol) [0x00007FF71C9C02C7]
	(No symbol) [0x00007FF71C9A493F]
	(No symbol) [0x00007FF71C97AAA7]
	(No symbol) [0x00007FF71C9BDEB7]
	(No symbol) [0x00007FF71C9A4563]
	(No symbol) [0x00007FF71C979FCE]
	(No symbol) [0x00007FF71C97918C]
	(No symbol) [0x00007FF71C979B81]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF71CD5DF54+1072532]
	(No symbol) [0x00007FF71C9F2D9C]
	Micro

## Propuestas de mejoras:
* Estaría bueno detectar fotos... son aquellas caras que no se modifican entre distintos screenshots.
* Estaría bueno registras las emociones detectadas durante toda la clase en un archivo plano para un análisis posterior del mismo.
* Podemos probar con los distintos métodos de detección de caras, esta seteado con "mtcnn", pero hay varias opciones:

    backends = [
    'opencv', 
    'ssd', 
    'dlib', 
    'mtcnn', 
    'fastmtcnn',
    'retinaface', 
    'mediapipe',
    'yolov8',
    'yunet',
    'centerface',
    ]

* Podemos guardar las caras detectadas para el cálculo del índice.

In [9]:
# Prueba detección de caras

if False:
    import cv2
    from deepface import DeepFace
    from collections import Counter

    # Cargar la imagen original
    #image_path = 'screenshots/test/test_zoom_meeting.jpg'
    #image_path = 'screenshots/test/prueba_clase.jpeg'
    image_path = 'screenshots/screenshot.png'
    image = cv2.imread(image_path)

    # Analizar las emociones en la imagen usando MTCNN para la detección facial
    resultados = DeepFace.analyze(image_path, detector_backend='mtcnn', actions=['emotion']) #yolov8 mtcnn
    #resultados = DeepFace.analyze(image_path, actions=['emotion'])

    # Dibujar rectángulos alrededor de las caras detectadas en una copia de la imagen original
    image_with_rectangles = image.copy()
    for i, face in enumerate(resultados):
        x, y, w, h = face['region']['x'], face['region']['y'], face['region']['w'], face['region']['h']
        cv2.rectangle(image_with_rectangles, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Obtener la emoción dominante
        dominant_emotion = face['dominant_emotion']
        
        # Crear la etiqueta
        label = f"Cara {i} - {dominant_emotion}"
        
        # Calcular la posición de la etiqueta
        label_position = (x, y - 10 if y - 10 > 10 else y + 10)
        
        # Colocar la etiqueta en la imagen
        cv2.putText(image_with_rectangles, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Guardar la imagen con los rectángulos dibujados
    cv2.imwrite('faces_detected.jpg', image_with_rectangles)
    
    print(f"\nCaras detectadas: {len(resultados)}\n")
    emotions = []
    for cara_nro, emotion_result in enumerate(resultados):
        emotion = emotion_result['emotion']
        strongest_emotion = max(emotion, key=emotion.get)
        score = emotion[strongest_emotion]
        print(f"* CARA {cara_nro}: Emoción más fuerte: {strongest_emotion}, Score: {score}")
        # Añadir la emoción a la lista
        dominant_emotion = emotion_result['dominant_emotion']
        emotions.append(dominant_emotion)
        
    engagement_score = compute_engagement(resultados)
    print(f"\nValor general del índice: {engagement_score}\n")
    
    emotion_counts = Counter(emotions)
    most_dominant_emotion = emotion_counts.most_common(1)[0][0]
    print(f"\nLa emoción más dominante es: {most_dominant_emotion}\n")

    # Mostrar la imagen con los rectángulos dibujados
    #cv2.imshow('Faces Detected', image_with_rectangles)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

1/1 [==============================] - 0s 25ms/step

Caras detectadas: 3

* CARA 0: Emoción más fuerte: fear, Score: 37.611329555511475
* CARA 1: Emoción más fuerte: angry, Score: 98.01923609009219
* CARA 2: Emoción más fuerte: fear, Score: 82.22355842590332

Valor general del índice: -1.0


La emoción más dominante es: fear

